# _Networking_ con Python

**Recuerda que al leer este documento directamente desde GitHub, no estás aprovechando su contenido al 100%.
Editando el código puedes aprender más que leyéndolo. Te recomendamos descargarlo y ejecutarlo desde tu computador para que puedas hacer modificaciones e interactuar con el código.**

En este _notebook_, veremos cómo implementar nuestros programas para que puedan comunicarse a través de la red.

## Arquitectura cliente-servidor

Esta arquitectura corresponde a un modelo de conexión entre máquinas, donde un servidor ofrece un servicio y los clientes consumen estos servicios. El rol del cliente es **iniciar la conexión al servidor**, y establecer comunicación con el servidor de acuerdo a los protocolos establecidos previamente. Por otro lado, el rol del servidor es **escuchar conexiones entrantes**; esto es, estar atento a las conexiones que intenten hacer los clientes, decidir si aceptar o no tal conexión, y entregar los servicios requeridos por el cliente. Ambas partes en la arquitectura cliente-servidor pueden aceptar conexiones del tipo TCP y UDP.

![Fuente: http://en.wikipedia.org/wiki/Client%E2%80%93server_model](imgs/client-server-model.png)
Fuente de la imagen: [Wikipedia](http://en.wikipedia.org/wiki/Client%E2%80%93server_model)

## _Sockets_

Un _**socket**_ es un objeto encargado de manejar la comunicación con un proceso corriendo en otra máquina, o en la misma máquina pero en otro puerto. Para crear un _socket_, primero necesitaremos especificar qué tipo de dirección IP otorgaremos (IPv4 o IPv6) y qué protocolo usaremos (TCP o UDP).

En Python, los _sockets_ son provistos por el módulo `socket`. Para crear un _socket_, debemos crear una instancia de la clase `socket(family, type)`, donde `family` es el tipo de dirección IP que usaremos, y `type` es el protocolo que utilizaremos. Algunos de los valores permitidos son:

- `family`
    - `AF_INET` para direcciones IPv4
    - `AF_INET6` para direcciones IPv6
- `type`
    - `SOCK_STREAM` para TCP
    - `SOCK_DGRAM` para UDP

Por ejemplo, para crear un _socket_ TCP con IPv4, lo hacemos de la siguiente manera.

In [15]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

### TCP

#### Cliente TCP

El cliente se conecta a un servidor y puerto dado con anterioridad. Para ello, debemos llamar al método `connect((host, port))` del _socket_. El método recibe una tupla, cuyo primer elemento debe ser una dirección IP acorde a lo que especificamos al momento de crear el _socket_, y el segundo elemento es el puerto al que nos queremos conectar. 

Opcionalmente, el primer elemento de la tupla puede ser un _string_ con el _hostname_, como por ejemplo, _python.org_. En ese caso, el sistema se encargará de averiguar la dirección IP para ese _hostname_.

Por ejemplo, hagamos un cliente que se conecte a `23.253.135.79` en el puerto 80 vía TCP.

In [16]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('23.253.135.79', 80))

Si nos queremos conectar a [python.org](https://www.python.org) también lo podemos hacer, sin conocer su dirección IP.

In [17]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('python.org', 80))

Llegó la hora de enviar algún mensaje. Los _sockets_ sólo transmiten _bytes_, por lo que cualquier cosa que enviemos debemos codificarla. Para enviar un mensaje, se puede ocupar el método `sendall(bytes)`.

In [18]:
sock.sendall('GET / HTTP/1.1\n\n\n'.encode('ascii'))

¿Qué nos habrá respondido el servidor? Para saberlo, debemos ocupar el método `recv(buffer)`, que nos retornará un objeto `bytes`. El parámetro que recibe este método es la cantidad máxima de _bytes_ que será leída. La documentación sugiere colocar una potencia de 2 pequeña, como 4.096.

In [19]:
data = sock.recv(4096)
print(data.decode('ascii'))

HTTP/1.1 301 Moved Permanently
Location: https://23.253.135.79/
Connection: Keep-Alive
Content-length: 0




El último paso, pero no el menos importante: **debemos cerrar la conexión**

In [20]:
sock.close()

Hagamos el ejercicio completo en una sola celda. Además, aprovechemos de manejar las excepciones pues, en _networking_, es muy probable que se produzcan errores.

In [21]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    sock.connect(('python.org', 80))
    sock.sendall('GET / HTTP/1.1\n\n\n'.encode('ascii'))
    data = sock.recv(4096)
    print(data.decode('ascii'))
except ConnectionError as e:
    print("Ocurrió un error.")
finally:
    # ¡No olvidemos cerrar la conexión!
    sock.close()

HTTP/1.1 301 Moved Permanently
Location: https://23.253.135.79/
Connection: Keep-Alive
Content-length: 0




#### Servidor TCP

El servidor debe escuchar las conexiones que le llegan a él a cierto puerto. Para ello, debemos "enlazar" un _socket_ a ese puerto con el método `bind((host, port))`. El primer elemento de la tupla es el _hostname_ de la máquina en que estamos corriendo el servidor; el segundo elemento es el puerto donde queremos escuchar las conexiones.

Supongamos que queremos escuchar en el puerto 9000.

In [ ]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina.
host = socket.gethostname()
port = 9000

sock.bind((host, port))

En el cliente no es necesario hacer un _bind_ entre el _host_ y el puerto, ya que el sistema operativo lo hace implícitamente a través del método `connect`, asignando al cliente un puerto aleatorio. Si el servidor al cual nos conectamos exigiera que el puerto de cada cliente esté en un rango específico, tendríamos que "enlazar" el cliente a un puerto específico también.

Ahora, empecemos a escuchar conexiones con el método `listen()`.

In [ ]:
sock.listen()

Ahora, empecemos a aceptar conexiones. Para ello, ocupamos el método `accept()`, que nos retornará un _socket_ con el que podremos comunicarnos con ese cliente en específico, y una dirección IP de ese cliente. El método esperará hasta que haya una conexión entrante; por lo tanto, si nadie se conecta, el programa quedará congelado.

In [ ]:
while True:
    socket_cliente, address = sock.accept()
    print("Conexión aceptada desde", address)
    socket_cliente.sendall("Gracias por conectarte\n".encode("ascii"))
    socket_cliente.close()

Cuando queramos terminar el programa servidor, debemos cerrar el _socket_ que estaba aceptando las conexiones.

In [ ]:
sock.close()

Hagamos el ejercicio completo en una sola celda. Supondremos que queremos que el servidor se cierre después de aceptar cinco clientes.

In [ ]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina.
host = socket.gethostname()
port = 9000

sock.bind((host, port))
sock.listen()

counter = 0
while counter < 5:
    try:
        socket_cliente, address = sock.accept()
        print("Conexión aceptada desde", address)
        socket_cliente.sendall("Gracias por conectarte\n".encode("ascii"))
        socket_cliente.close()
        counter += 1
    except ConnectionError:
        print("Ocurrió un error.")

sock.close()

El siguiente código corresponde a un posible cliente para el servidor implementado en la celda anterior.

In [ ]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Obtenemos el nombre de la máquina a la que nos queremos conectar.
# En este caso, nos queremos conectar a esta misma máquina.
host = socket.gethostname()
port = 9000

sock.connect((host, port))
data = sock.recv(4096)
print(data.decode("ascii"))
print("lol")
sock.close() 

### UDP

#### Cliente UDP

Dado que el protocolo UDP no establece una conexión, la comunicación UDP es mucho más simple de implementar. Por ejemplo, para enviar un mensaje como cliente a un servidor simplemente ocupamos el método `sendto(mensaje, (host_destino, puerto_destino))` como se muestra a continuación.

In [ ]:
import socket

server_host = socket.gethostname()
server_port = 15000

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
mensaje = "Hola, simplemente te estoy enviando un mensaje.".encode('ascii')
sock.sendto(mensaje, (server_host, server_port))

Opcionalmente, podemos recibir información enviada de vuelta. El metodo `recvfrom(buffer)` retorna, además de los datos, la dirección y puerto desde donde fueron enviados. Recordemos que el parámetro `buffer` es la cantidad máxima de _bytes_ que será leída, que se recomienda que sea una potencia de 2 pequeña.

In [ ]:
data, (host_origen, puerto_origen) = sock.recvfrom(4096)
print(data.decode('utf-8'))

Cuando terminemos de ocupar el _socket_, debemos cerrarlo.

In [ ]:
sock.close()

#### Servidor UDP

Si de forma similar queremos implementar un servidor que envía mensajes usando el protocolo UDP, simplemente debemos preocuparnos de responder a la misma dirección desde donde se nos ha enviado algún mensaje. Por ejemplo, el siguiente código podría representar al servidor que se comunica con el cliente implementado anteriormente.

In [ ]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Con un string vacío como argumento del bind, indicamos que el socket es 
# alcanzable desde cualquier dirección que pueda tener el servidor.
sock.bind(("", 15000))

while True:
    data, (host_cliente, puerto_cliente) = sock.recvfrom(4096)
    print(data.decode('ascii'))
    respuesta = f"Aquí va mi respuesta para {host_cliente}."
    sock.sendto(respuesta.encode('utf-8'), (host_cliente, puerto_cliente))